# Graph Construction: Graph Tool
In this Notebook, we construct the graph of the Rt's Users during the Paro Nacional. 

In [1]:
import pickle
from glob import glob
import numpy as np
import pandas as pd
import graph_tool.all as gt
from datetime import datetime
from tqdm import tqdm
from scipy.sparse import csr_matrix
from scipy.sparse import lil_matrix

# Daily graphs
Each graph corresponds to the RT Network of each day between April 28 of 2021 and June 30 of 2021